In [ ]:
import numpy as np
from numpy import load
from matplotlib import pyplot as plt
import matplotlib
import  os

In [ ]:
root_dir="~/Downloads/"

In [ ]:
base_dir = root_dir + 'Segmented Numpy Data/'

In [ ]:
#prepare the data from the images we cut

data_output = np.zeros((1,6443008))
data_input = np.zeros((1,6443008)) 
all_files = sorted(os.listdir(base_dir))

for i in range(0,174):
    filename = all_files[i]
    if filename.endswith("seg.npz"): 
      dict_data1 = load(base_dir + filename)
      data_temp1 = dict_data1['arr_0']
      data_temp1 = data_temp1.reshape(1,6443008)
      data_output = np.vstack((data_output,data_temp1))
    else:
      dict_data2 = load(base_dir + filename)
      data_temp2 = dict_data2['arr_0']
      data_temp2 = data_temp2.reshape(1,6443008)
      data_input = np.vstack((data_input,data_temp2))
        
data_output = np.delete(data_output,0, axis = 0)
data_input = np.delete(data_input,0,axis = 0)
print(data_output.shape)
print(data_input.shape)

In [ ]:
# Reshape into original input output of 176*208*176
#Here 87 is initial sample size for both inputs and outputs
data_input2 = data_input.reshape(87,176,208,176)
data_output2 = data_output.reshape(87,176,208,176)
print(data_output2.shape)
print(data_input2.shape)

In [ ]:
#remove outliers

data_input2[data_input2 < 0] = 0
data_input2[data_input2 > 255] = 255

data_output2[data_output2 < 0] = 0
data_output2[data_output2 > 255] = 255

In [ ]:
#find the boundary of ventricles in x, y, z axis in each image
index = np.zeros((87,6))

# x-axis
for i in range(0,87):
  for j in range(0,88):
    if (np.sum(data_output2[i,j,:,:])) > 0:
      if j > 0:
        index[i,0] = j
        break

for i in range(0,87):
  for k in range(175,88,-1):
    if (np.sum(data_output2[i,k,:,:])) > 0:
      if k < 175:
        index[i,1] = k
        break
        
# y-axis
for i in range(0,87):
  for j in range(0,104):
    if (np.sum(data_output2[i,:,j,:])) > 0:
      if j > 0:
        index[i,2] = j
        break

for i in range(0,87):
  for k in range(175,88,-1):
    if (np.sum(data_output2[i,:,k,:])) > 0:
      if k < 175:
        index[i,3] = k
        break

#z-axis
for i in range(0,87):
  for j in range(0,88):
    if (np.sum(data_output2[i,:,:,j])) > 0:
      if j > 0:
        index[i,4] = j
        break

for i in range(0,87):
  for k in range(175,88,-1):
    if (np.sum(data_output2[i,:,:,k])) > 0:
      if k < 175:
        index[i,5] = k
        break

In [ ]:
print(data_input2.shape)
print(data_output2.shape)

# The common boundaries are the following
# x-axis (42,121)
# y-axis (40,161)
# z-axis (32,134)

#  Adjust a few more or less pixels to fit a good dimension shape 

# Extract the part include ventricles

data_input_cut = data_input2[:,42:122,37:165,20:148]
data_output_cut = data_output2[:,42:122,37:165,20:148]
print(data_input_cut.shape)
print(data_output_cut.shape)

#Final Dimension should be 80, 128,128

In [ ]:
#Check whether data has been imported properly
# Plotting first image 

f, ax = plt.subplots(1,3)

ax[0].imshow(data_input_cut[0,50,:,:],cmap="Greys")
ax[0].imshow(np.ma.masked_array(data_output_cut[0,50,:,:], data_output_cut[0,50,:,:]==0.0))

ax[1].imshow(data_input_cut[0,:,50,:],cmap="Greys")
ax[1].imshow(np.ma.masked_array(data_output_cut[0,:,50,:], data_output_cut[0,:,50,:]==0.0))

ax[2].imshow(data_input_cut[0,:,:,50],cmap="Greys")
ax[2].imshow(np.ma.masked_array(data_output_cut[0,:,:,50], data_output_cut[0,:,:,50]==0.0))

In [ ]:
#To increase sample size flip the data along axis 1 and 2 , further concatenate it 
data_input_cut_flipud = np.flip(data_input_cut, axis=1)
data_output_cut_flipud = np.flip(data_output_cut, axis=1)

data_input_cut_fliplr = np.flip(data_input_cut, axis=2)
data_output_cut_fliplr = np.flip(data_output_cut, axis=2)

data_input_cut_all = np.concatenate((data_input_cut, data_input_cut_fliplr), axis=0)
data_output_cut_all = np.concatenate((data_output_cut, data_output_cut_fliplr), axis=0)

#Final sample size (261,80,128,128)

In [ ]:
# Get the data into X and Y dataset arrays
X_dataset=data_input_cut_all
Y_dataset=data_output_cut_all
print(X_dataset.shape)
print(Y_dataset.shape)

In [ ]:
#Randomize train test split and set seed for reproducibility
np.random.seed(0)
random_train = np.sort(np.random.choice(87,70,replace=False))
random_test = np.setdiff1d(np.array([i for i in range(87)]),random_train,assume_unique=False)

X_train = X_dataset[random_train]
X_test = X_dataset[random_test]

Y_train = Y_dataset[random_train]
Y_test = Y_dataset[random_test]

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
# reshaping data into input tensor shape acceptable in keras 3D conv-net
X_train = X_train.reshape(-1, 80, 128,128, 1)
X_test = X_test.reshape(-1, 80, 128,128, 1)
Y_train = Y_train.reshape(-1, 80, 128,128, 1)
Y_test = Y_test.reshape(-1, 80, 128,128, 1)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)